# NUTNR Calibration Parser
The ISUS and SUNA are

In [1]:
import csv
import re
import os
import numpy as np
import pandas as pd

In [2]:
from utils import *

In [3]:
os.getcwd()

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review'

In [4]:
def get_calibration_files(serial_nums, cal_directory):
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)
        sn = str(sn[0])
        files = []
        for file in os.listdir(cal_directory):
            if 'Calibration_File' in file:
                if sn in file:
                    files.append(file)
        calibration_files.update({uid:files})
    return calibration_files

In [5]:
calibration_files = {}
for uid in serial_nums.keys():
    sn = serial_nums.get(uid)
    sn = sn[0]
    sn = str(sn)
    files = []
    for file in os.listdir(cal_directory):
        if 'Calibration_File' in file:
            if sn in file:
                print(file)

NameError: name 'serial_nums' is not defined

In [6]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Results/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/NUTNRB/'

In [7]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [8]:
NUTNR = whoi_asset_tracking(spreadsheet=excel_spreadsheet,sheet_name=sheet_name,instrument_class='NUTNR',series='B')

In [9]:
NUTNR

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
876,NUTNR,B,239,115084,A00065,CGINS-NUTNRB-00239,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00004\n3305-00108-00048\n3305-00108...,3305-00308-00001,3305-00508-00040,3305-00900-00075\n3305-00900-00144\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,GI01SUMO-00001\nCP04OSSM-00006,NaN,NSIF,Reading High nitrate levels during calibration...
877,NUTNR,B,240,115085,A00066,CGINS-NUTNRB-00240,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00003\n3305-00108-00029\n3305-00108...,NaN,NaN,3305-00900-00008\n3305-00900-00231\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,CP01CNSM-00001\nGS01SUMO-00002,NaN,NSIF,"09/2017: Clock issue - resets back to Jan 1, 2..."
878,NUTNR,B,260,115671,A00383,CGINS-NUTNRB-00260,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00010\n3305-00108-00056,NaN,3305-00508-00010,3305-00900-00109\n3305-00900-00317,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI Spare\nCP1 spare\nGS01SUMO-00001\nCP04OSSM-...,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
879,NUTNR,B,261,115672,A00384,CGINS-NUTNRB-00261,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00011\n3305-00108-00021\n3305-00108...,3305-00308-00007\n3305-00308-00031\n3305-00308...,3305-00508-00022\n3305-00508-00041,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,CP03ISSM-00002\nCP01CNSM-00005\nCP01CNSM-00007,NaN,NSIF,Returned to vendor 6/24/14 (RMA#2014-125)\nSen...
880,NUTNR,B,262,115673,A00385,CGINS-NUTNRB-00262,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00012\n3305-00108-00064,3305-00308-00002,NaN,3305-00900-00064\n3305-00900-00153\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI01SUMO-00001,NaN,NaN,Sent to vedor as part of trade in for new (SUN...
881,NUTNR,B,266,116564,A00880,CGINS-NUTNRB-00266,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00015\n3305-00108-00040\n3305-00108...,3305-00308-00004\n3305-00308-00032,3305-00508-00024,3305-00900-00036\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP04OSSM-00001\nCP04OSSM-00004\nCP01CNSM-00008,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
882,NUTNR,B,267,116562,A00878,CGINS-NUTNRB-00267,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00018\n3305-00108-00053,NaN,3305-00508-00009,3305-00900-00109\n3305-00900-00363,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,GS01SUMO-00001\nCP01CNSM-00006,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
883,NUTNR,B,268,116563,A00879,CGINS-NUTNRB-00268,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00017\n3305-00108-00046\n3305-00108...,3305-00308-00003\n3305-00308-00034,3305-00508-00023,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP03ISSM-00001\nCP03ISSM-00004\nCP03ISSM-00006,NaN,NSIF,4/2016: Scheduled to sample every half hour. S...
884,NUTNR,B,269,116565,A00881,CGINS-NUTNRB-00269,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00016\n3305-00108-00047\n3305-00114...,3305-00308-00005\n3305-00308-00009,3305-00508-00001,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP01CNSM-00002\nCP01CNSM-00003\nCP03ISSM-00005,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
885,NUTNR,B,270,116899,A01142,CGINS-NUTNRB-00270,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00020\n3305-00108-00050,3305-00308-00006\n3305-00308-00008,3305-00508-00038,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-09-24 00:00:00,CP3a Spare\nCP04OSSM-00002\nGI01SUMO-00003,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...


**Identify the QCT Testing documents associated with each individual instrument (the UID)**

In [10]:
uids = list(set(NUTNR['UID']))
qct_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    NUTNR['UID_match'] = NUTNR['UID'].apply(lambda x: True if uid in x else False)
    qct_series = NUTNR[NUTNR['UID_match'] == True]['QCT Testing']
    qct_series = list(qct_series.iloc[0].split('\n'))
    qct_dict.update({uid:qct_series})

In [11]:
qct_dict

{'CGINS-NUTNRB-00277': ['3305-00108-00028', '3305-00108-00079'],
 'CGINS-NUTNRB-00239': ['3305-00108-00004',
  '3305-00108-00048',
  '3305-00108-00062'],
 'CGINS-NUTNRB-01064': ['3305-00127-00009', '3305-00127-00035'],
 'CGINS-NUTNRB-00270': ['3305-00108-00020', '3305-00108-00050'],
 'CGINS-NUTNRB-01090': ['3305-00127-00016'],
 'CGINS-NUTNRB-01092': ['3305-00127-00015'],
 'CGINS-NUTNRB-01106': ['3305-00127-00020'],
 'CGINS-NUTNRB-01062': ['3305-00127-00007'],
 'CGINS-NUTNRB-01063': ['3305-00127-00008', '3305-00127-00037'],
 'CGINS-NUTNRB-00268': ['3305-00108-00017',
  '3305-00108-00046',
  '3305-00108-00070'],
 'CGINS-NUTNRB-00285': ['3305-00108-00035', '3305-00108-00063'],
 'CGINS-NUTNRB-00283': ['3305-00108-00033', '3305-00108-00055'],
 'CGINS-NUTNRB-00262': ['3305-00108-00012', '3305-00108-00064'],
 'CGINS-NUTNRB-00276': ['3305-00108-00027', '3305-00108-00081'],
 'CGINS-NUTNRB-00240': ['3305-00108-00003',
  '3305-00108-00029',
  '3305-00108-00080',
  '3305-00108-00087'],
 'CGINS-NUT

**Get the serial numbers for each NUTNR, and use those serial numbers to search for and return all of the relevant vendor documents for a particular instrument.**

In [12]:
serial_nums = get_serial_nums(NUTNR, uids)
serial_nums

{'CGINS-NUTNRB-00277': [277],
 'CGINS-NUTNRB-00239': [239],
 'CGINS-NUTNRB-01064': ['NTR-1064'],
 'CGINS-NUTNRB-00270': [270],
 'CGINS-NUTNRB-01090': ['NTR-1090'],
 'CGINS-NUTNRB-01092': ['NTR-1092'],
 'CGINS-NUTNRB-01106': ['NTR-1106'],
 'CGINS-NUTNRB-01062': ['NTR-1062'],
 'CGINS-NUTNRB-01063': ['NTR-1063'],
 'CGINS-NUTNRB-00268': [268],
 'CGINS-NUTNRB-00285': [285],
 'CGINS-NUTNRB-00283': [283],
 'CGINS-NUTNRB-00262': [262],
 'CGINS-NUTNRB-00276': [276],
 'CGINS-NUTNRB-00240': [240],
 'CGINS-NUTNRB-01089': ['NTR-1089'],
 'CGINS-NUTNRB-01065': ['NTR-1065'],
 'CGINS-NUTNRB-00274': [274],
 'CGINS-NUTNRB-01086': ['NTR-1086'],
 'CGINS-NUTNRB-00267': [267],
 'CGINS-NUTNRB-00273': [273],
 'CGINS-NUTNRB-00260': [260],
 'CGINS-NUTNRB-01102': ['NTR-1102'],
 'CGINS-NUTNRB-01087': ['NTR-1087'],
 'CGINS-NUTNRB-01103': ['NTR-1103'],
 'CGINS-NUTNRB-00275': [275],
 'CGINS-NUTNRB-00261': [261],
 'CGINS-NUTNRB-00280': [280],
 'CGINS-NUTNRB-00271': [271],
 'CGINS-NUTNRB-00266': [266],
 'CGINS-NUTNRB-0

In [13]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

{'CGINS-NUTNRB-00277': ['NUTNR-B_ISUS_SN_277_Calibration_Files_2015-06-18.zip',
  'NUTNR-B_ISUS_SN_277_Calibration_Files_2017-06-06.zip'],
 'CGINS-NUTNRB-00239': ['NUTNR-B_ISUS_SN_239_Calibration_Files _2012-11-30.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2012-10-08.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2016-02-29.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2016-09-30.zip'],
 'CGINS-NUTNRB-01064': ['NUTNR-B_SUNA_SN_NTR-1064_Calibration_Files_2017-09-05.zip',
  'NUTNR-B_SUNA_SN_NTR-1064_Calibration_Files_2018-12-19.zip'],
 'CGINS-NUTNRB-00270': ['NUTNR-B_ISUS_SN_270_Calibration_Files_2014-10-01.zip',
  'NUTNR-B_ISUS_SN_270_Calibration_Files_2016-02-29.zip'],
 'CGINS-NUTNRB-01090': ['NUTNR-B_SUNA_SN_NTR-1090_Calibration_Files_2017-10-24.zip'],
 'CGINS-NUTNRB-01092': ['NUTNR-B_SUNA_SN_NTR-1092_Calibration_Files_2017-10-16.zip'],
 'CGINS-NUTNRB-01106': ['NUTNR-B_SUNA_SN_NTR-1106_Calibration_Files_2017-11-28.zip'],
 'CGINS-NUTNRB-01062': ['NUTNR-B_SUNA_SN_NTR-1062_Calibrati

In [70]:
pre_dict

{'CGINS-NUTNRB-00277': nan,
 'CGINS-NUTNRB-00239': ['3305-00308-00001'],
 'CGINS-NUTNRB-01064': nan,
 'CGINS-NUTNRB-00270': ['3305-00308-00006', '3305-00308-00008'],
 'CGINS-NUTNRB-01090': nan,
 'CGINS-NUTNRB-01092': nan,
 'CGINS-NUTNRB-01106': nan,
 'CGINS-NUTNRB-01062': nan,
 'CGINS-NUTNRB-01063': nan,
 'CGINS-NUTNRB-00268': ['3305-00308-00003', '3305-00308-00034'],
 'CGINS-NUTNRB-00285': ['3305-00308-00033', '3305-00308-00067'],
 'CGINS-NUTNRB-00283': ['3305-00308-00012', '3305-00308-00048'],
 'CGINS-NUTNRB-00262': ['3305-00308-00002'],
 'CGINS-NUTNRB-00276': ['3305-00308-00065'],
 'CGINS-NUTNRB-00240': nan,
 'CGINS-NUTNRB-01089': ['3305-00327-00055'],
 'CGINS-NUTNRB-01065': nan,
 'CGINS-NUTNRB-00274': ['3305-00308-00011',
  '3305-00308-00012',
  '3305-00308-00013'],
 'CGINS-NUTNRB-01086': ['3305-00327-00036'],
 'CGINS-NUTNRB-00267': nan,
 'CGINS-NUTNRB-00273': ['3305-00308-00066'],
 'CGINS-NUTNRB-00260': nan,
 'CGINS-NUTNRB-01102': ['3305-00327-00051'],
 'CGINS-NUTNRB-01087': ['330

In [71]:
file_dict = {}
for uid in sorted(uids):
    cal_files = cal_dict[uid]
    

{'CGINS-NUTNRB-00277': ['NUTNR-B_ISUS_SN_277_Calibration_Files_2015-06-18.zip',
  'NUTNR-B_ISUS_SN_277_Calibration_Files_2017-06-06.zip'],
 'CGINS-NUTNRB-00239': ['NUTNR-B_ISUS_SN_239_Calibration_Files _2012-11-30.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2012-10-08.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2016-02-29.zip',
  'NUTNR-B_ISUS_SN_239_Calibration_Files_2016-09-30.zip'],
 'CGINS-NUTNRB-01064': ['NUTNR-B_SUNA_SN_NTR-1064_Calibration_Files_2017-09-05.zip',
  'NUTNR-B_SUNA_SN_NTR-1064_Calibration_Files_2018-12-19.zip'],
 'CGINS-NUTNRB-00270': ['NUTNR-B_ISUS_SN_270_Calibration_Files_2014-10-01.zip',
  'NUTNR-B_ISUS_SN_270_Calibration_Files_2016-02-29.zip'],
 'CGINS-NUTNRB-01090': ['NUTNR-B_SUNA_SN_NTR-1090_Calibration_Files_2017-10-24.zip'],
 'CGINS-NUTNRB-01092': ['NUTNR-B_SUNA_SN_NTR-1092_Calibration_Files_2017-10-16.zip'],
 'CGINS-NUTNRB-01106': ['NUTNR-B_SUNA_SN_NTR-1106_Calibration_Files_2017-11-28.zip'],
 'CGINS-NUTNRB-01062': ['NUTNR-B_SUNA_SN_NTR-1062_Calibrati

**Identify the calibration csvs stored in asset management which correspond to a particular instrument.**

In [14]:
csv_dict = load_asset_management(NUTNR, asset_management_directory)
for uid in sorted(csv_dict):
    sn = serial_nums.get(uid)
    sn = str(sn[0])
    for file in sorted(csv_dict.get(uid)):
        print(sn + ' ' + file)

239 CGINS-NUTNRB-00239__20140910.csv
239 CGINS-NUTNRB-00239__20160930.csv
240 CGINS-NUTNRB-00240__20131121.csv
240 CGINS-NUTNRB-00240__20151214.csv
240 CGINS-NUTNRB-00240__20170615.csv
240 CGINS-NUTNRB-00240__20180531.csv
260 CGINS-NUTNRB-00260__20150218.csv
260 CGINS-NUTNRB-00260__20161012.csv
261 CGINS-NUTNRB-00261__20150509.csv
261 CGINS-NUTNRB-00261__20160513.csv
261 CGINS-NUTNRB-00261__20161110.csv
262 CGINS-NUTNRB-00262__20140910.csv
266 CGINS-NUTNRB-00266__20141212.csv
266 CGINS-NUTNRB-00266__20160527.csv
267 CGINS-NUTNRB-00267__20150218.csv
268 CGINS-NUTNRB-00268__20141214.csv
268 CGINS-NUTNRB-00268__20160528.csv
268 CGINS-NUTNRB-00268__20161110.csv
269 CGINS-NUTNRB-00269__20141213.csv
269 CGINS-NUTNRB-00269__20150507.csv
269 CGINS-NUTNRB-00269__20161011.csv
270 CGINS-NUTNRB-00270__20150509.csv
270 CGINS-NUTNRB-00270__20160710.csv
271 CGINS-NUTNRB-00271__20150815.csv
271 CGINS-NUTNRB-00271__20161007.csv
272 CGINS-NUTNRB-00272__20150315.csv
272 CGINS-NUTNRB-00272__20161027.csv
2

In [68]:
uids = list(set(NUTNR['UID']))
pre_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    NUTNR['UID_match'] = NUTNR['UID'].apply(lambda x: True if uid in x else False)
    pre_series = NUTNR[NUTNR['UID_match'] == True]['PreDeployment']
    try:
        pre_series = list(pre_series.iloc[0].split('\n'))
    except:
        pre_series = np.nan
    pre_dict.update({uid:pre_series})

In [214]:
uids = list(set(NUTNR['UID']))
post_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    NUTNR['UID_match'] = NUTNR['UID'].apply(lambda x: True if uid in x else False)
    post_series = NUTNR[NUTNR['UID_match'] == True]['Post Deployment']
    try:
        post_series = list(post_series.iloc[0].split('\n'))
    except:
        post_series = np.nan
    post_dict.update({uid:post_series})

In [76]:
predeployment = {}
for file in os.listdir(qct_directory):
    if '00308' in file and file.endswith('.zip'):
        filepath = '/'.join((qct_directory,file))
        with ZipFile(filepath) as zfile:
            filename = [name for name in zfile.namelist() if name.lower().endswith('.cal')]
            predeployment.update({file: filename})
    
    if '00327' in file and file.endswith('.zip'):
        filepath = '/'.join((qct_directory,file))
        with ZipFile(filepath) as zfile:
            filename = [name for name in zfile.namelist() if name.lower().endswith('.cal')]
            predeployment.update({file: filename})

In [150]:
postdeployment = {}
for file in os.listdir(qct_directory):
    if '00508' in file and file.endswith('.zip'):
        filepath = '/'.join((qct_directory,file))
        with ZipFile(filepath) as zfile:
            filename = [name for name in zfile.namelist() if name.lower().endswith('.cal')]
            postdeployment.update({file: filename})
    
    if '00527' in file and file.endswith('.zip'):
        filepath = '/'.join((qct_directory,file))
        with ZipFile(filepath) as zfile:
            filename = [name for name in zfile.namelist() if name.lower().endswith('.cal')]
            postdeployment.update({file: filename})

In [129]:
qct = {}
for file in os.listdir(qct_directory):
    if '00108' in file and file.endswith('.zip'):
        filepath = '/'.join((qct_directory,file))
        with ZipFile(filepath) as zfile:
            filename = [name for name in zfile.namelist() if name.lower().endswith('.cal')]
            qct.update({file: filename})
    
    if '00127' in file and file.endswith('.zip'):
        filepath = '/'.join((qct_directory,file))
        with ZipFile(filepath) as zfile:
            filename = [name for name in zfile.namelist() if name.lower().endswith('.cal')]
            qct.update({file: filename})

In [149]:
for file in os.listdir(qct_directory):
    if '127' in file and '.doc' not in file:
        filepath = qct_directory + file
        with open(filepath,errors='ignore') as x:
            data = x.read()
        if 'reference' in data:
            print(file)

In [131]:
for q in sorted(qct.keys()):
    print(q + ': ' + str(sorted(qct[q])))

3305-00127-00002-A.zip: []
3305-00127-00003-A.zip: []
3305-00127-00004-A.zip: []
3305-00127-00005-A.zip: []
3305-00127-00006-A.zip: []
3305-00127-00007-A.zip: []
3305-00127-00008-A.zip: []
3305-00127-00009-A.zip: []
3305-00127-00010-A.zip: []
3305-00127-00011-A.zip: []
3305-00127-00012-A.zip: []
3305-00127-00013-A.zip: []
3305-00127-00014-A.zip: []
3305-00127-00015-A.zip: []
3305-00127-00016-A.zip: []
3305-00127-00017-A.zip: []
3305-00127-00018-A.zip: []
3305-00127-00019-A.zip: []
3305-00127-00020-A.zip: []
3305-00127-00021-A.zip: []
3305-00127-00022-A.zip: []
3305-00127-00023-A.zip: []
3305-00127-00024-A.zip: ['SNA1115D.cal']
3305-00127-00025-A.zip: ['SNA1127C.cal']
3305-00127-00026-A.zip: []
3305-00127-00027-A.zip: []
3305-00127-00028-A.zip: []
3305-00127-00029-A.zip: []
3305-00127-00030-A.zip: []
3305-00127-00031-A.zip: []
3305-00127-00032-A.zip: []
3305-00127-00033-A.zip: []
3305-00127-00034-A.zip: []
3305-00127-00035-A.zip: []
3305-00127-00036-A.zip: []
3305-00127-00037-A.zip: []


In [77]:
for pre in sorted(predeployment.keys()):
    print(pre + ': ' + str(predeployment[pre]))

3305-00308-00001-A.zip: ['ISUS239D.CAL', 'ISUS239C.CAL']
3305-00308-00002-A.zip: ['ISUS262B.CAL']
3305-00308-00003-A.zip: ['ISUS268B.CAL']
3305-00308-00004-A.zip: ['ISUS266D.CAL']
3305-00308-00005-A.zip: ['ISUS269C.CAL']
3305-00308-00006-A.zip: ['ISUS270B.CAL']
3305-00308-00007-A.zip: ['ISUS261E.CAL']
3305-00308-00008-A.zip: ['ISUS270J.CAL']
3305-00308-00009-A.zip: ['ISUS269D.CAL']
3305-00308-00010-A.zip: ['ISUS271C.CAL']
3305-00308-00011-A.zip: ['ISUS274B.CAL']
3305-00308-00012-A.zip: ['ISUS283B.CAL']
3305-00308-00013-A.zip: ['ISUS284B.CAL']
3305-00308-00014-A.zip: []
3305-00308-00015-A.zip: ['ISUS254C_2014-10-01.CAL']
3305-00308-00016-A.zip: ['ISUS244E.CAL', 'ISUS244C.CAL', 'ISUS244D.CAL']
3305-00308-00017-A.zip: ['ISUS255B.CAL']
3305-00308-00018-A.zip: ['ISUS253F.CAL', 'ISUS253C.CAL', 'ISUS253D.CAL', 'ISUS253E.CAL']
3305-00308-00019-A.zip: ['ISUS256D.CAL']
3305-00308-00020-A.zip: ['ISUS263C.CAL']
3305-00308-00021-A.zip: ['ISUS265C.CAL', 'ISUS265D.CAL']
3305-00308-00022-A.zip: ['ISUS

In [128]:
calibration = {}
for uid in sorted(cal_dict.keys()):
    calibration.update({uid:[]})
    for file in cal_dict[uid]:
        filepath = '/'.join((cal_directory,file))
        with ZipFile(filepath) as zfile:
            filename = [name for name in zfile.namelist() if name.lower().endswith('.cal') and 'ZCoef' not in name]
            for fname in filename:
                calibration[uid].append(fname)
for key in sorted(calibration.keys()):
    for file in sorted(calibration.get(key)):
        print(file)

Calibration Files/ISS0239I.CAL
ISS0239A.cal
ISS0239e.cal
ISUS239B.CAL
ISUS240B.CAL
ISUS240B.CAL
ISUS240C.cal
ISUS240E.CAL
ISUS240E.CAL
ISS0260E.cal
ISS0261D.cal
ISS0261G.cal
ISS0261K.CAL
ISS0262C.cal
ISS0262C.cal
ISS0266I.cal
NUTNR_ISUS_SN_266_Calibration_Files_2014-06-28/ISUS266B.CAL
ISUS267F.CAL
NUTNR_ISUS_SN_267_Calibration_Files_2014-06-28/ISS0267A.cal
ISS0268D.cal
ISS0268G.cal
NUTNR_ISUS_SN_268_Calibration_Files_2014-06-28/ISS0268A.cal
ISS0269F.cal
NUTNR_ISUS_SN_269_Calibration_Files_2014-06-28/ISUS269B.CAL
ISS0270A.cal
ISS0270K.cal
ISS0271B.cal
ISS0271F.CAL
ISS0272A.cal
ISS0272D.cal
ISS0273A.cal
ISS0273D.cal
ISS0273I.CAL
ISS0274A.cal
ISS0274D.cal
ISS0275A.cal
ISS0275C.cal
ISUS275B.CAL
ISS0276A.cal
ISS0276B.CAL
ISS0277A.cal
ISS0277B.cal
ISS0280A.cal
ISUS280G.CAL
ISS0283A.cal
ISS0283D.cal
ISS0284A.cal
ISS0284D.cal
Calibration Files/ISS0285D.CAL
ISS0285A.cal
SNA1063B.cal
SNA1064C.cal
SNA1065A.cal
SNA1086B.cal
SNA1087B.cal
SNA1088B.cal
SNA1088C.CAL
SUNA1089B.cal
SNA1090B.cal
SNA1091B

========================================================================================================================
**Now, I have all the available ISUS and SUNA .CAL files available in Vault and/or on Asset Management. This includes Calibration files, QCT files, Pre-deployment files, and Post-deployment files. Next, I need to generate files for each deployment.**

========================================================================================================================
**Now, need to get all the files for a particular NUTNR UID:**

In [195]:
uid = sorted(uids)[0]
uid

'CGINS-NUTNRB-00239'

**Get and return the full directory paths to the relevant files**

In [196]:
csv_files = csv_dict[uid]
csv_path = []
for cf in csv_files:
    path = generate_file_path(asset_management_directory, cf)
    csv_path.append(path)
csv_path

['/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/NUTNRB/CGINS-NUTNRB-00239__20160930.csv',
 '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/NUTNRB/CGINS-NUTNRB-00239__20140910.csv']

In [197]:
cal_files = cal_dict[uid]
cal_path = []
for cf in cal_files:
    path = generate_file_path(cal_directory, cf)
    cal_path.append(path)
cal_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files _2012-11-30.zip',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files_2012-10-08.zip',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files_2016-02-29.zip',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files_2016-09-30.zip']

In [210]:
pre_files = pre_dict[uid]
pre_path = []
for pf in pre_files:
    path = generate_file_path(qct_directory, pf, ext=['.zip'])
    pre_path.append(path)
pre_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Results/3305-00308-00001-A.zip']

In [216]:
post_files = post_dict[uid]
post_path = []
for pf in post_files:
    path = generate_file_path(qct_directory, pf, ext=['.zip'])
    post_path.append(path)
post_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Results/3305-00508-00001-A.zip']

========================================================================================================================
## Metadata Comparison
Now the goal is to compare the calibration csvs contained in asset management against the calibration coefficients stored in the QCT files.

In [217]:
import shutil

In [259]:
class NUTNRCalibration():
    # Class that stores calibration values for CTDs.

    def __init__(self, uid):
        self.serial = None
        self.uid = uid
        self.notes = {}
        self.coefficients = {
            'CC_cal_temp':[],
            'CC_di':[],
            'CC_eno3':[],
            'CC_eswa':[],
            'CC_lower_wavelength_limit_for_spectra_fit':'217',
            'CC_upper_wavelength_limit_for_spectra_fit':'240',
            'CC_wl':[]
        }
        self.date = {}       

    @property
    def uid(self):
        return self._uid

    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self._uid = d
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")
            

    def parse_cal(self, filepath):
        """
        Function to parse the NUTNR calibration file.
        
        Args:
            filepath - the full filepath to the calibration
                file to be parsed. Zip files are acceptable.
        Calls:
            parse_ISUS - function to parse the calibration 
                file for an ISUS
            parse_SUNA - function to parse the calibration
                file for a SUNA
        Returns:
            self.coefficients - a dictionary object with the
                parsed NUTNR calibration coefficients
        """
        
        data = self.open_cal(filepath)
        
        # Check on if the data is an ISUS or SUNA 
        if 'ISUS' in filepath:
            self.parse_ISUS(data)
        else:
            self.parse_SUNA(data)
            

    def open_cal(self, filepath):
        """
        Function that opens and reads in cal file
        information for a NUTNR. Zipfiles are acceptable inputs.
        """
        
        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                # Check if ISUS or SUNA to get the appropriate name
                if 'ISUS' in filepath:
                    filename = [name for name in zfile.namelist() 
                                if name.lower().endswith('.cal') and 'is' in name.lower()]
                else:
                    filename = [name for name in zfile.namelist() 
                                if name.lower().endswith('.cal') and name.lower().startswith('sna')]
                # Get and open the latest calibration file
                if len(filename) == 1:
                    data = zfile.read(filename[0]).decode('ascii')
                elif len(filename) > 1:
                    filename = max(filename)
                    data = zfile.read(filename[0]).decode('ascii')
                else:
                    FileExistsError(f"No .cal file found in {filepath}")
                        
        elif filepath.lower().endswith('.cal'):
            if 'isus' or 'suna' in filepath.lower().split('/')[-1]:
                with open(filepath) as file:
                    data = file.read()
        else:
            pass
        
        return data
        

    def parse_ISUS(self, data):
        """
        Function to parse the ISUS NUTNR calibration file. Similar to
        but not interchangeable with the SUNA parser. 
        """
        
        for line in data.splitlines():
    
            if line.startswith('H'):
                _, info, *ignore = line.split(',')
                info = info.split()
        
                if info[0] == 'ISUS':
                    self.serial = info[1].lstrip('0')
            
                if info[0] == 'T_CAL_SWA':
                    self.coefficients['CC_cal_temp'] = info[1]
                    
                if 'File creation time' in line:
                    _, _, _, date, time = line.split()
                    cal_date = pd.to_datetime(date + ' ' + time).strftime('%Y%m%d')
                    self.date.update({len(self.date): cal_date})
                    
            elif line.startswith('E'):
                _, wl, eno3, eswa, _, di = line.split(',')
        
                # Append the new data
                self.coefficients['CC_wl'].append(wl)
                self.coefficients['CC_eno3'].append(eno3)
                self.coefficients['CC_eswa'].append(eswa)
                self.coefficients['CC_di'].append(di)
                
            else:
                pass
        
    
    def parse_SUNA(self, data):
        """
        Function to parse the SUNA NUTNR calibration file. Similar to
        but not interchangeable with the ISUS parser.
        """
        
        for line in data.splitlines():
    
            if line.startswith('H'):
                _, info, *ignore = line.split(',')
                info = info.split()
        
                if info[0] == 'SUNA':
                    self.serial = info[1].lstrip('0')
            
                if info[0] == 'T_CAL_SWA':
                    self.coefficients['CC_cal_temp'] = info[1]
                    
                if 'File creation time' in line:
                    _, _, _, date, time = line.split()
                    cal_date = pd.to_datetime(date + ' ' + time).strftime('%Y%m%d')
                    self.date.update({len(self.date): cal_date})
                    
            elif line.startswith('E'):
                _, wl, eno3, eswa, _, di = line.split(',')
        
                # Append the new data
                self.coefficients['CC_wl'].append(float(wl))
                self.coefficients['CC_eno3'].append(float(eno3))
                self.coefficients['CC_eswa'].append(float(eswa))
                self.coefficients['CC_di'].append(float(di))
                
            else:
                pass

            
    def write_csv(self, outpath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.

        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the CTD object's coefficient dictionary
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is
                written to the specified directory from the outpath.
        """

        # Run a check that the coefficients have actually been loaded
        if len(self.coefficients) == 0:
            raise ValueError('No calibration coefficients have been loaded.')

        # Create a dataframe to write to the csv
        data = {
            'serial': [self.serial]*len(self.coefficients),
            'name': list(self.coefficients.keys()),
            'value': list(self.coefficients.values())
        }
        df = pd.DataFrame().from_dict(data)

        # Define a function to reformat the notes into an uniform system
        def reformat_notes(x):
            # First, get rid of 
            try:
                np.isnan(x)
                x = ''
            except:
                x = str(x).replace('[','').replace(']','')
            return x
        
        # Now merge the coefficients dataframe with the notes
        if len(self.notes) > 0:
            notes = pd.DataFrame().from_dict({
                'name':list(self.notes.keys()),
                'notes':list(self.notes.values())
            })
            df = df.merge(notes, how='outer', left_on='name', right_on='name')
        else:
            df['notes'] = ''
        
        # Sort the data by the coefficient name
        df = df.sort_values(by='name')

        # Generate the csv name
        cal_date = max(self.date.values())
        csv_name = self.uid + '__' + cal_date + '.csv'

        # Write the dataframe to a csv file
        # check = input(f"Write {csv_name} to {outpath}? [y/n]: ")
        check = 'y'
        if check.lower().strip() == 'y':
            df.to_csv(outpath+'/'+csv_name, index=False)

**First, need to copy the calibration csvs from asset management to the local temp directory.**

In [261]:
#shutil.rmtree('/'.join((os.getcwd(),'temp')))

In [262]:
temp_directory = '/'.join((os.getcwd(),'temp',uid))
ensure_dir(temp_directory)

In [263]:
for file in csv_path:
    savedir = '/'.join((os.getcwd(),'temp',uid,'csv'))
    ensure_dir(savedir)
    shutil.copy(file, savedir)

In [264]:
os.listdir('/'.join((os.getcwd(),'temp')))

['CGINS-NUTNRB-00269']

In [265]:
'/'.join((os.getcwd(),'temp',uid,'csv'))

'/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Metadata_Review/temp/CGINS-NUTNRB-00269/csv'

**Next, write all the QCT files to the temp directory in the appropriate csv format. This will print out any QCT files which don't parse.**

**Answer: The NUTNRs don't have QCTs which print out the calibration files.**

In [266]:
cal_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files _2012-11-30.zip',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files_2012-10-08.zip',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files_2016-02-29.zip',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Cal/NUTNR-B_ISUS_SN_239_Calibration_Files_2016-09-30.zip']

In [267]:
max(nutnr.date.values())

'20121106'

In [268]:
for path in cal_path:
    nutnr = NUTNRCalibration(uid)
    savedir = '/'.join((os.getcwd(),'temp',uid,'cal'))
    ensure_dir(savedir)
    nutnr.parse_cal(path)
    filesource = path.split('/')[-1]
    nutnr.notes.update({'CC_cal_temp':f'Source for calibration is {filesource}.'})
    nutnr.write_csv(savedir)

In [269]:
os.listdir(savedir)

['CGINS-NUTNRB-00269__20121106.csv',
 'CGINS-NUTNRB-00269__20160930.csv',
 'CGINS-NUTNRB-00269__20160201.csv',
 'CGINS-NUTNRB-00269__20120920.csv']

In [ ]:
savedir = '/'.join((os.getcwd(),'temp',uid,'cal'))
ensure_dir(savedir)
nutnr.parse_cal(cal_path[0])
nutnr.write_csv(savedir)

In [241]:
nutnr.coefficients

{'CC_cal_temp': [],
 'CC_di': [],
 'CC_eno3': [],
 'CC_eswa': [],
 'CC_lower_wavelength_limit_for_spectra_fit': '217',
 'CC_upper_wavelength_limit_for_spectra_fit': '240',
 'CC_wl': []}

In [ ]:
os.listdir(savedir)

In [ ]:
CAL = pd.read_csv('/'.join((os.getcwd(),'temp','cal','CGINS-NUTNRB-00240__20120920.csv')))
CAL['value'] = CAL['value'].apply(lambda x: x.strip("[]").replace("'",'').split(', '))
CAL['value'] = CAL['value'].apply(lambda x: pd.to_numeric(x))

In [ ]:
os.listdir('/'.join((os.getcwd(),'temp','csv')))

In [ ]:
CSV = pd.read_csv('/'.join((os.getcwd(),'temp','csv','CGINS-NUTNRB-00240__20131121.csv')))
CSV['value'] = CSV['value'].apply(lambda x: x.strip("[]").replace("'",'').split(', '))
CSV['value'] = CSV['value'].apply(lambda x: pd.to_numeric(x))

In [ ]:
CAL

In [ ]:
CSV

In [ ]:
check = {}
for ind in CSV.index:
    a = CSV['value'].loc[ind]
    b = CAL['value'].loc[ind]
    c = np.isclose(a, b, rtol=1e-8, atol=1e-15)
    if any(c) is False:
        check.update({CSV['name'].loc[ind]: c})
    else:
        check.update({CSV['name'].loc[ind]: 'No differences.'})
    

In [ ]:
check

In [ ]:
nutnr.date

========================================================================================================================
### Compare results
Now, with QCT files parsed into csvs which follow the UFrame format, I can load both the QCT and the calibratoin csvs into pandas dataframes, which will allow element by element comparison in relatively few lines of code.

In [ ]:
def get_file_date(x):
    x = str(x)
    ind1 = x.index('__')
    ind2 = x.index('.')
    return x[ind1+2:ind2]

In [ ]:
# Now we want to compare dataframe
csv_files = pd.DataFrame(sorted(os.listdir('temp/csv')),columns=['csv'])
csv_files['cal date'] = csv_files['csv'].apply(lambda x: get_file_date(x))
csv_files.set_index('cal date',inplace=True)
csv_files

In [ ]:
cal_files = pd.DataFrame(sorted(os.listdir('temp/cal')),columns=['cal'])
cal_files['cal date'] = cal_files['cal'].apply(lambda x: get_file_date(x))
cal_files.set_index('cal date',inplace=True)
cal_files

In [ ]:
df_files = csv_files.join(cal_files,how='outer').fillna(value='-999')
df_files

In [ ]:
a = 'temp/csv/' + 'CGINS-NUTNRB-00239__20160930.csv'
b = 'temp/csv/' + 'CGINS-NUTNRB-00239__20160929.csv'

In [ ]:
os.listdir('temp/csv/')

In [ ]:
shutil.copy(a,b)

In [ ]:
os.listdir('temp/csv')

In [ ]:
!rm 'temp/csv/CGINS-NUTNRB-00239__20160930.csv'

In [ ]:
csv_files = pd.DataFrame(sorted(os.listdir('temp/csv')),columns=['csv'])
csv_files['cal date'] = csv_files['csv'].apply(lambda x: get_file_date(x))
csv_files.set_index('cal date',inplace=True)
csv_files

In [ ]:
a = 'CGINS-NUTNRB-00239__20120920.csv'
apath = '/'.join((temp_directory,'csv',a))
bpath = '/'.join((temp_directory,'cal',a))

In [ ]:
CSV = pd.read_csv(apath)
CAL = pd.read_csv(bpath)

In [ ]:
CSV

In [ ]:
CAL

In [ ]:
CSV['name'] == CAL['name']

In [ ]:
for i in CAL['value'].loc[1]:
    print(i)

In [ ]:
def convert_type(x):
    for i,j in enumerate(x):
        x[i] = float(j)
    return x
        

In [ ]:
def check_cal_coeffs(coeffs_dict):
    
    # Part 1: coeff by coeff comparison between each source of coefficients
    keys = list(coeffs_dict.keys())
    comparison = {}
    for i in range(len(keys)):
        names = (keys[i], keys[i - (len(keys)-1)])
        check = len(coeffs_dict.get(keys[i])['value']) == len(coeffs_dict.get(keys[i - (len(keys)-1)])['value'])
        if check:
            compare = np.isclose(coeffs_dict.get(keys[i])['value'], coeffs_dict.get(keys[i - (len(keys)-1)])['value'])
            comparison.update({names:compare})
        else:
            pass
        
    # Part 2: now do a logical_and comparison between the results from part 1
    keys = list(comparison.keys())
    i = 0
    mask = comparison.get(keys[i])
    while i < len(keys)-1:
        i = i + 1
        mask = np.logical_and(mask, comparison.get(keys[i]))
        print(i)
       
    return mask 

In [ ]:
result = {}
for cal_date in df_files.index:
    # Part 1, load all of the csv files
    coeffs_dict = {}
    for source,fname in df_files.loc[cal_date].items():
        if fname != '-999':
            load_directory = '/'.join((os.getcwd(),'temp',source,fname))
            df_coeffs = pd.read_csv(load_directory)
            for i in list(set(df_coeffs['serial'])):
                print(source + '-' + fname + ': ' + str(i))
            df_coeffs.set_index(keys='name',inplace=True)
            df_coeffs.sort_index(inplace=True)
            coeffs_dict.update({source:df_coeffs})
        else:
            pass
    
    # Part 2, now check the calibration coefficients
    mask = check_cal_coeffs(coeffs_dict)
    
    # Part 3: get the calibration coefficients are wrong
    # and show them
    fname = df_files.loc[cal_date]['csv']
    if fname == '-999':
        incorrect = 'No csv file.'
    else:
        incorrect = coeffs_dict['csv'][mask == False]
    result.update({fname:incorrect})